In [ ]:
#| default_exp scripts

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2

In [ ]:
#| include: false
from nbdev import nbdev_export
nbdev_export()

In [ ]:
#| exporti

import argparse
import yaml
import os
import time

from pangraph_constructor.graph import GenomeGraph
from pangraph_constructor.export import exportProject

In [ ]:
#| export
def pantograph_script():
    parser = argparse.ArgumentParser(prog='pantograph',
                                     description='This is command line tool giving access to the library called pangraph_constructor for manipulating and \
                                     visualision pre-processing of genome graphs.')
    
    subparsers = parser.add_subparsers(help='Multiple commands available')
    
    export_parser = subparsers.add_parser('export-vis',help='Export a set of genome graphs (project) to a project directory structure to be visualised in \
                                                             Pantorgaph visualisation component.')
    
    export_parser.add_argument('file',help='YAML configuration file for exporting process.')
    export_parser.add_argument('-g','--generate',action='store_true',help='Generate model commented configuration file at the given path.')
    
    export_parser.set_defaults(func=export_project_script)
    
    ann2graph_parser = subparsers.add_parser('annotation2graph',help='Generate gene graph (as opposed to nucleotide graph) from a set of annotations for multiple \
                                                                    accessions and multiple molecules/chromosomes.')
    ann2graph_parser.add_argument('file',help='YAML configuration file for graph generation from annotation.')
    ann2graph_parser.add_argument('-g','--generate',action='store_true',help='Generate model commented configuration file at the given path.')
    
    ann2graph_parser.set_defaults(func=ann2graph_script)
    
    paths2graph_parser = subparsers.add_parser('paths2graph', help='Generate graph from path file(s).')
    
    paths2graph_parser.add_argument('file',help='YAML configuration file for graph generation from paths.')
    paths2graph_parser.add_argument('-g','--generate',action='store_true',help='Generate model commented configuration file at the given path.')
    
    paths2graph_parser.set_defaults(func=paths2graph_script)
    
    sortgraph_parser = subparsers.add_parser('sort-graph',help='Perform graph sorting to get preferred order of nodes for linearised visualisation.')
    
    sortgraph_parser.add_argument('--quiet','-q',action='store_true', help='Suppress most of output. False (i.e. verbose) is not set.')
    
    sortgraph_parser.add_argument('--isseq','-s',action='store_true', help='Does this graph contains nucleotide sequences. False is not set.')
    
    sortgraph_parser.add_argument('--output','-o', action='store', help='File path where to save sorted graph. If not set, the input will be overwritten.')
    
    sortgraph_parser.add_argument('input', help='Relative (to current directory) or absolute path to the GFA file with the graph to be sorted.')
    
    sortgraph_parser.set_defaults(func=sort_graph_script)
    
    args = parser.parse_args()
        
    if 'func' in vars(args):
        args.func(args)
    else:
        parser.print_help()
    
    
    

In [ ]:
#| export
def paths2graph_script(args):
    
    if 'file' not in vars(args):
        raise AttributeError('Positional argument with path to setting YAML file has to be provided.')
        
    if args.generate:
        defaultYAML = """\
#####################################################################################
# This is example YAML setting file generated for pantograph paths2graph tool.      #
# Paths are stubs and should be substituted!!!                                      #
#                                                                                   #
# There are two versions of path files:                                             #
# - v1: Each chromosome/molecule is a separate file, each row of the file has       #
#       format as follows                                                           #
#       <accession ID>: <node1><dir1>,<node2><dir2>,...                             #
#       <dirX> is a directionality of each node and can be '+' or '-'.              #
# - v2: One path file contains all chromosomes/molecules, each row of the file      #
#       has the following format:                                                   #
#       <accession ID>-<chromosome number>: <node1><dir1>,<node2><dir2>,...         #
#       Here <chromosome number> can be any id, which will be added to 'Chr' in     #
#       all names and IDs.
# There is a possibility not to have directionality. In this case, parameter 
# `directional` should be set to false.
#####################################################################################

# `pathfileDir` common path to all path files (like path to your project files)
pathfileDir: /path/to/pathfiles

# `pathFiles` is a dictionary of chromosomes as keys and values with paths (relative to `pathFileDir`) to path files (for v1 path files), or
# `pathFile` is a string containing a single path (relative to `pathFileDir`) to path file (for v2 path file).
# Only one options should be used! If both options are used, pathFile option will be ignorred!
# If you need to process a single path file for a single molecule (v1), then use pathFiles and provide 
# a list of one file.
# pathFiles:
#     Chr1: path/to/pathfile1.txt
#     Chr2: path/to/pathfile2.txt
pathFile: path/to/pathfile.txt

# `outputpath` is an absolute path (directory) where to save the resulting graphs
outputpath: /path/to/graphs

# `outputBasename` is a base for creating graph file names in the following format:
# - for v1 <outputBasename>_<chromosome name>.gfa
#   here <chromosome name is taken from keys for `pathFiles`
# - for v2 <outputBasename>_Chr<chromosome number>.gfa
#   here <chromosome number> is taken from the file itself (see file structure above).
outputBasename: 'path_graph'

# The following parameters are optional:

# `sort` defines how the paths (accessions should be sorted). There are three options:
# 1) `sort` can be true or false. If it is false, then no sorting happens and accessions will be sorted in the order
#    as they appear in pathFile(s). If they appear differently for different chromosomes, then order of accessions
#    in graphs will be different. If it is true, then they will sorted in alphabethical order.
# 2) `sort` is a string, it should contain one accession name (e.g. reference accession) and it will be placed on top
#    with the rest of the accessions sorted in alphabetical order.
# 3) `sort` is list of accession names. In this case, the provided order will be used for ordering paths. If some 
#    accessions will be missing from the list (even if they present in path file(s)), they will be omitted from graphs.
#
# If `sort` is not provided, default is true.
# sort: false
# sort: 'reference'
# sort:
#     - accession3
#     - accession2
#     - accession1

# `directional` is a boolean (true or false) and indicate whether the tool should look for directionality
# indication in the file next to node (true) or it is not there and positive directionality should be assumed for all nodes (false).
# Default is true.
# directional: false
"""

        with open(args.file,'w') as f:
            f.write(defaultYAML)
        return
    
    with open(args.file,'r') as f:
        params = yaml.load(f,Loader=yaml.Loader)
    
    # Change code to use params read from YAML file.
    
    pathfileDir = params['pathfileDir'].rstrip('/').rstrip('\\')
    
    directional = params.get('directional', True)
    sort_acc = params.get('sort', True)
    
    outputpath = params["outputpath"].rstrip("/").rstrip("\\")
    
    outputbase = f'{outputpath}{os.path.sep}{params[outputBasename]}'
    
    if pathFiles in params:
    
        # For v1
        for seqName,pathfile in params['pathFiles'].items(): 

            paths = pathFileToPathDict(f'{pathfileDir}{os.path.sep}{pathfile}',directional,sort_acc)

            # paths = {}
            # for accession in sorted(list(_paths.keys())):
            #     paths[accession] = _paths[accession]
            # del _paths

            coregraph = GenomeGraph(pathsDict=paths)

            # On undirected coregraph sorting is not optimal! Check sorting!!!

            coregraph.treeSort()

            if len(coregraph.nodes)!=len(coregraph.order):
                print('Sorting failed and not all nodes were sorted. Saving unsorted graph')
                output = f'{outputbase}_{seqName}_unordered.gfa'
                coregraph.order = list(range(1,len(coregraph.nodes)+1))
                coregraph.toGFA(output,doSeq=False)
            else:
                coreGFApath = f'{outputbase}_{seqName}.gfa'
                coregraph.toGFA(coreGFApath,False)
                
    elif pathFile in params:
        
        # For v2
        pathfileDir = '../../1001G/coreGraph/new_Nov2022/'

        pathsfile = f'paths.txt'

        paths = pathFileToPathDict(f'{pathfileDir}{os.path.sep}{params["pathFile"]}',directional,sort_acc,True)

        for seqNum in paths.keys():#range(1,6): 
            # paths = {}
            # for accession in sorted(list(_paths.keys())):
            #     paths[accession] = _paths[accession]
            # del _paths

            coregraph = GenomeGraph(pathsDict=paths[seqNum])

            # On undirected coregraph sorting is not optimal! Check sorting!!!

            coregraph.treeSort()

            if len(coregraph.nodes)!=len(coregraph.order):
                print('Sorting failed and not all nodes were sorted. Saving unsorted graph')
                output = f'{outputbase}_Chr{seqNum}_unordered.gfa'
                coregraph.order = list(range(1,len(coregraph.nodes)+1))
                coregraph.toGFA(output,doSeq=False)
            else:
                coreGFApath = f'{outputbase}_Chr{seqNum}.gfa'
                coregraph.toGFA(coreGFApath,False)
                
    else:
        raise AttributeError('Either `pathFiles` or `pathFile` should be provided in setting file.')
        

In [ ]:
#| export
def ann2graph_script(args):
    if 'file' not in vars(args):
        raise AttributeError('Positional argument with path to setting YAML file has to be provided.')
        
    if args.generate:
        defaultYAML = """\
#####################################################################################
# This is example YAML setting file generated for pantograph annotation2graph tool. #
# Paths are stubs and should be substituted!!!                                      #
#####################################################################################

# `graphpath` is full path including filename where the graph should be saved. If extension is not provided, gfa will be added.
# If `chromosomes` are provided, then the actual files will get chromosomes names at the end of file name (before extension), e.g.
# graphpath=/path/to/graph.gfa and chromosomes contains 'chr1', then final file will be /path/to/graph_chr1.gfa
graphpath: /path/to/graph.gfa

# `annotationFilesPaths` common path to all files (like path to your project files)
annotationFilesPaths: /path/to/project

# `annotationFiles` is a list of paths (relative to `annotationFilesPaths`) to annotation (gff3) files, 
# This is required parameter!
annotationFiles:
    - 'path/to/file1.gff'
    - 'path/to/file2.gff'

# The following parameters are optional:

# `sequenceFilesDict` is a dictionary, where keys are IDs of accessions used in annotation files and value
# is a path to FASTA file (relative to `annotationFilesPaths`). Assumption is that FASTA sequence names
# are the same as GFF3 sequence names.
# sequenceFilesDict:
#     accession1: path/to/fileA.fasta
#     accession2: path/to/fileB.fasta

# `pangenomeFiles` is a list of GFF3 file paths (relative to `annotationFilesPaths`) for the same intervals as in `annotationFiles`, 
# ID fields in GFF2 metadata should be the same.
# pangenomeFiles:
#     - 'path/to/file1.gff'
#     - 'path/to/file2.gff'


# `accOrder` is a list which gives an order of accessions in the graph. If not given, accessions will be sorted in 
# alphabetical order.
accOrder:
    - accession2
    - accession1

# `chromosomes` a list of molecules/chromosomes, if provided, the tool will create only one graph for each chromosome, otherwise, it will create one graph for 
# all chromosomes (exporting of such graphs to Pantograph Visualisation is not currently implemented).
# chromosomes: 
#    - 'Chr1'
#    - 'Chr2'
#    - 'Chr3'

# `doUS` (default: False) if set, tool will add unrelated sequence blocks between annotated genes/blocks. Not recommended.
# doUS: true

# `refAnnotationFile` is a path (relative to `annotationFilesPaths`) to gff3 file with reference notation for gene names. 
# In main annotations reference gene names should be identified either in "gene" records under "AT" key (prioritised), 
# or in "mRNA" records under "Name" key (fallback). If `transmapFile` is provided, it will be used to find corresponding 
# similarity IDs and reference genes IDs.
# refAnnotationFile: path/to/reference.gff

# `refAccession` is an Accession ID for reference annotation (should be provided if `refAnnotationFile` if provided).
refAccession: 'reference_accession'

# `refSequenceFile` is a FASTA path (relative to `annotationFilesPaths`), where sequences of each block/gene for reference.
# refSequenceFile: path/to/reference.fasta

# `transmapFile` is a tab delimited file with column "Orthogroup", which contains similarity ID for genes and a column with name
# given by `transmapFileRefCol` which contains reference annotation gene names. There should be either both or none provided.
# transmapFile: path/to/transmap.map
# transmapFileRefCol: 'reference'
"""

        with open(args.file,'w') as f:
            f.write(defaultYAML)
        return
    
    with open(args.file,'r') as f:
        params = yaml.load(f,Loader=yaml.Loader)
    
    commonpath = params.pop('annotationFilesPaths').rstrip('/').rstrip('\\')
    
    graphpath = params.pop('graphpath')
    gpath,gext = os.path.splitext(graphpath)
    if gext=='':
        gext = 'gfa'
    
    params['annotationFiles'] = [f'{commonpath}{os.path.sep}{p}' for p in params['annotationFiles']]
    
    if 'sequenceFilesDict' in params:
        params['sequenceFilesDict'] = {k:f'{commonpath}{os.path.sep}{v}' for k,v in params['sequenceFilesDict']}
    
    if 'pangenomeFiles' in params:
        params['pangenomeFiles'] = [f'{commonpath}{os.path.sep}{p}' for p in params['pangenomeFiles']]
        
    if 'refAnnotationFile' in params:
        params['refAnnotationFile'] = f'{commonpath}{os.path.sep}{params["refAnnotationFile"]}'
    
    if 'refSequenceFile' in params:
        params['refSequenceFile'] = f'{commonpath}{os.path.sep}{params["refSequenceFile"]}'
    
    if 'transmapFile' in params:
        params['transmapFile'] = f'{commonpath}{os.path.sep}{param["transmapFile"]}'
    
    
    
    if 'chromosomes' in params:
        chrs = params.pop('chromosomes')
        
        for chromosome in chrs:
            print(f'\nProcessing {chromosome}\n============')

            curtst = time.time()
            
            graph = GenomeGraph(**params, chromosome = chromosome)

            print(f'Generating graph for {chromosome} took {time.time() - curtst} seconds')
            
            
            
            curtst = time.time()
            graph.treeSort()
            print(f'Sorting graph for {chromosome} took {time.time() - curtst} seconds')
            if len(graph.nodes)!=len(graph.order):
                print('Sorting failed and not all nodes were sorted. Saving unsorted graph')
                output = f'{gpath}_{chromosome}_unsorted.{gext}'
                graph.order = list(range(1,len(graph.nodes)+1))
                graph.toGFA(output,doSeq=False)
            else:
                output = f'{gpath}_{chromosome}.{gext}'
                graph.toGFA(f'{gfadir}{os.path.sep}{gfaFilename}',doSeq=False)
    else:
        print(f'\nProcessing single graph for all chromosomes\n============')

        curtst = time.time()
        
        graph = GenomeGraph(**params)

        print(f'Generating graph for all chromosomes took {time.time() - curtst:.0f} seconds')
        
        curtst = time.time()
        graph.treeSort()
        print(f'Sorting graph for all chromosomes took {time.time() - curtst:.0} seconds')
        
        if len(graph.nodes)!=len(graph.order):
            print('Sorting failed and not all nodes were sorted. Saving unsorted graph')
            output = f'{gpath}_unordered.{gext}'
            graph.order = list(range(1,len(graph.nodes)+1))
            graph.toGFA(output,doSeq=False)
        else:
            output = f'{gpath}.{gext}'
            graph.toGFA(output,doSeq=False)
    # change function call here.
    #exportProject(**params)

In [ ]:
#| export
def sort_graph_script(args):
    inputfile = args.input
    
    vargs = vars(args)
    isseq = args.get('isseq',False)
    
    outputfile = vargs.get('output', inputfile)
    
    quiet = vargs.get('quiet',False)
    
    graph_new = GenomeGraph(gfaPath=inputfile,isGFASeq=isseq,quiet=quiet)

    graph_new.treeSort()

    assert len(graph_new.nodes)==len(graph_new.order)

    graph_new.toGFA(outputfile,doSeq=isseq)

In [ ]:
#| export
def export_project_script(args):
    if 'file' not in vars(args):
        raise AttributeError('Positional argument with path to setting YAML file has to be provided.')
    
    print(args)
    
    if args.generate:
        defaultYAML = """\
###############################################################################
# This is example YAML setting file generated for pantograph export-vis tool. #
# Paths are stubs and should be substituted!!!                                #
###############################################################################

# projectID - this is a project id and will be the name of directory that holds all data to visualise the project.
#             It is recommended not to have special symbols and spaces in this name
# REQUIRED
projectID: pangenome_project

# projectName - Human readable short name describing the project. It should not be more than 50 symbols.
# REQUIRED
projectName: "All chromosomes for 27 accessions of species X"

# caseDict - a dictionary of each individual graph within project. Usually one file per chromosome, although, in some cases 
#            there can be only one graph for all chromosomes or other cases.
#            Graph names can contain path relative to `pathToGraphs` if they are in different directories.
# REQUIRED
caseDict:
    Chr1: pangenome_graph_chromosome1.gfa
    Chr2: pangenome_graph_chromosome2.gfa

# pathToIndex - along with project data structure, exporting adds information to the index.json file which is
#               read by Pantograph visualisation component in the first instance. If it does not exist at the given path
#               it will be created at the given path.
# REQUIRED
pathToIndex: /path/to/pantograph/visualisation/data/root/

# pathToGraphs - path where graph files mentioned in `caseDict` can be found.
# REQUIRED
pathToGraphs: /path/to/all/graph/files/

# When graphs are exported some metadata (annotations, genome and pangenome positions) can be recorded to Redis DB
# If Redis DB is not available or recording of metadata is not needed, then parameter redisHost should be omitted.
# Otherwise, if Redis DB is available and metadata should be recorded, then redisHost should be set to the hostname (or IP address)
# of the Redis DB server
# redisHost: 'redis.example.com'
# By default, this tool will use port 6379 for Redis DB server and DB ID 0 (zero). If any of these should be changed, then set relevant variables as follows:
# redisPort: 6379
# redisDB: 0

# Suffix is used for the naming of the output directories for each graph (molecule or chromosome), e.g. if project is called 'pangenome_project', 
# case is 'chr1' and suffix is '_new', then the output directory for this graph visualisation data structure will be called 'pangenome_project_chr1_new'
# If not set, then no suffix will be added.
# suffix: ''

# maxLengthComponent defined how large (in visual columns) can the component be. If continuity allows for longer components, it will be broken
# into components of maximum `maxLengthComponent` columns. If not set, default of 100 is used.
# maxLengthComponent: 100

# maxLengthChunk defined how many components will be recorded into a single file in the visualisation data structure. Smaller chunk files allows to
# load data into memory more precisely, but if a lot of small components should be loaded, larger chunks make it quicker.
# If not set, default of 6 will be used.
# maxLengthChunk: 6,

# Each cell has calculated inversion ratio (see user manual for details). It will be shown as inverted (shades of red) if its inversion ratio
# is more than `inversionThreshold` set
# If not set, default is 0.5
# inversionThreshold: 0.5,

# If the graph is not nucleotide graph (e.g. gene graph generated from paths or annotations), `isSeq` should be set to False.
# Default is True
# isSeq: False,

# Which zoom levels should be generated. The number indicate how many maximum nucleotides (or genes in case of gene graph) can be displayed as
# single column in the visualised matrix. Zoom level of 1 is added automatically if it is not in the list. Each previous zoom level should be 
# a factor or the next one, i.e. next zoom level should be obtained by multiplying previous one by an integer more than 1.
# E.g. zoom levels [1,2,4,8] is correct, but zoom levels [1,2,4,6] is incorrect because 4 is not a factor of 6.
# Defaults is [1], meaning to generate only lowest zoom level (single nucleotide/gene resolution).
# zoomLevels: [1,2,4]

# If `fillZoomLevel` set to True, then on top of the highest zoom level set in `zoomLevels`, the function will keep adding more zoom levels
# by multiplying it by 2 until there is no non-linear links to remove left.
# Default is True
# fillZoomLevel: False
        
        """
        
        with open(args.file,'w') as f:
            f.write(defaultYAML)
        return
    
    with open(args.file,'r') as f:
        params = yaml.load(f,Loader=yaml.Loader)
    exportProject(**params)
    